# HerdNet Training Pipeline

Pipeline completo de entrenamiento de HerdNet siguiendo la metodología de Delplanque et al. (2023):

1. **Fase 0**: Generación de parches de entrenamiento y validación
2. **Fase 1**: Entrenamiento inicial (Stage 1) sobre parches
3. **Fase 2**: Generación de Hard Negative Patches (HNPs)
4. **Fase 3**: Entrenamiento con HNPs (Stage 2)
5. **Fase 4**: Evaluación final sobre imágenes completas


## Imports


In [ ]:
from pathlib import Path
from shutil import copy2
import json

import pandas as pd
from dataclasses import asdict

from utils.herdnet import (
    TrainConfig,
    train_stage1,
    train_stage2,
    HNPConfig,
    generate_hard_negative_patches,
    EvalConfig,
    evaluate_full_images,
    evaluate_points_from_csv,
)

from utils.rf_detr import generate_patch_dataset, PatchSummary


## Configuración Global


In [ ]:
# Configuración de paths
DATA_ROOT = Path("data-delplanque")
OUTPUT_ROOT = Path("outputs/herdnet")

# Configuración de patches
PATCH_SIZE = 512
PATCH_OVERLAP = 160
MIN_VISIBILITY = 0.1

# Configuración de entrenamiento
BATCH_SIZE = 4
NUM_WORKERS = 4
EPOCHS_STAGE1 = 100
EPOCHS_STAGE2 = 50
LR_STAGE1 = 1e-4
LR_STAGE2 = 1e-6

# Configuración de evaluación
MATCH_RADIUS = 5.0
STITCH_OVERLAP = 160

# WandB (opcional)
WANDB_PROJECT = None  # "herdnet-training"
WANDB_ENTITY = None
WANDB_MODE = "disabled"  # "online" para activar


# Fase 0 — Generación de Parches

Dividimos las imágenes de alta resolución (24MP) en parches de 512×512 píxeles para facilitar el entrenamiento.


In [ ]:
patch_jobs = [
    {
        "split": "train",
        "images_dir": DATA_ROOT / "train",
        "json_file": DATA_ROOT / "train.json",
        "output_dir": OUTPUT_ROOT / "patches" / "train",
        "patch_width": PATCH_SIZE,
        "patch_height": PATCH_SIZE,
        "overlap": PATCH_OVERLAP,
        "min_visibility": MIN_VISIBILITY,
    },
    {
        "split": "val",
        "images_dir": DATA_ROOT / "val",
        "json_file": DATA_ROOT / "val.json",
        "output_dir": OUTPUT_ROOT / "patches" / "val",
        "patch_width": PATCH_SIZE,
        "patch_height": PATCH_SIZE,
        "overlap": PATCH_OVERLAP,
        "min_visibility": MIN_VISIBILITY,
    },
]


In [ ]:
patch_summaries = []

for job in patch_jobs:
    print(f"\n{'='*70}")
    print(f"Generando parches: {job['split']}")
    print(f"{'='*70}")
    
    summary = generate_patch_dataset(
        images_dir=job["images_dir"],
        json_file=job["json_file"],
        output_dir=job["output_dir"],
        patch_width=job["patch_width"],
        patch_height=job["patch_height"],
        overlap=job["overlap"],
        min_visibility=job["min_visibility"],
        include_background_category=True,
    )
    
    entry = {"split": job["split"]}
    entry.update(asdict(summary))
    patch_summaries.append(entry)
    
    print(f"✓ Parches creados: {summary.patches_created}")
    print(f"✓ Anotaciones: {summary.annotations_patches}")

pd.DataFrame(patch_summaries)


## Convertir anotaciones COCO a CSV

HerdNet requiere formato CSV con columnas: `images`, `x`, `y`, `labels`.


In [ ]:
def coco_to_csv(coco_json: Path, output_csv: Path) -> pd.DataFrame:
    """Convierte anotaciones COCO a formato CSV para HerdNet."""
    with coco_json.open("r") as f:
        coco = json.load(f)
    
    # Mapear image_id a file_name
    image_map = {img["id"]: img["file_name"] for img in coco["images"]}
    
    # Extraer anotaciones
    records = []
    for ann in coco["annotations"]:
        x, y, w, h = ann["bbox"]
        cx, cy = x + w / 2, y + h / 2
        records.append({
            "images": image_map[ann["image_id"]],
            "x": cx,
            "y": cy,
            "labels": ann["category_id"],
        })
    
    df = pd.DataFrame(records)
    output_csv.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_csv, index=False)
    print(f"✓ Guardado CSV: {output_csv} ({len(df)} anotaciones)")
    return df


In [ ]:
# Convertir train patches
train_csv = coco_to_csv(
    OUTPUT_ROOT / "patches" / "train" / "_annotations.coco.json",
    OUTPUT_ROOT / "patches" / "train" / "gt.csv",
)

# Convertir val patches
val_csv = coco_to_csv(
    OUTPUT_ROOT / "patches" / "val" / "_annotations.coco.json",
    OUTPUT_ROOT / "patches" / "val" / "gt.csv",
)


# Fase 1 — Entrenamiento Stage 1

Entrenar HerdNet sobre los parches generados. Este es el entrenamiento inicial sin Hard Negative Patches.


In [ ]:
stage1_config = TrainConfig(
    train_root=OUTPUT_ROOT / "patches" / "train",
    train_csv=OUTPUT_ROOT / "patches" / "train" / "gt.csv",
    val_root=OUTPUT_ROOT / "patches" / "val",
    val_csv=OUTPUT_ROOT / "patches" / "val" / "gt.csv",
    work_dir=OUTPUT_ROOT / "stage1",
    epochs=EPOCHS_STAGE1,
    batch_size=BATCH_SIZE,
    learning_rate=LR_STAGE1,
    num_workers=NUM_WORKERS,
    patch_size=PATCH_SIZE,
    stitch_overlap=STITCH_OVERLAP,
    wandb_project=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY,
    wandb_mode=WANDB_MODE,
    wandb_run_name="herdnet_stage1",
)


In [ ]:
print("\n" + "="*70)
print("INICIANDO ENTRENAMIENTO STAGE 1")
print("="*70 + "\n")

stage1_result = train_stage1(stage1_config)

print("\n" + "="*70)
print("STAGE 1 COMPLETADO")
print("="*70)
print(f"✓ Best checkpoint: {stage1_result.best_checkpoint}")
print(f"✓ Latest checkpoint: {stage1_result.latest_checkpoint}")
print("="*70 + "\n")


# Fase 2 — Generación de Hard Negative Patches

Usar el modelo de Stage 1 para generar predicciones sobre las imágenes de entrenamiento completas y extraer parches de falsos positivos.


## Preparar CSV de imágenes completas

Necesitamos un CSV con las anotaciones de las imágenes completas de entrenamiento (no los parches).


In [ ]:
# Convertir anotaciones de train (imágenes completas) a CSV
train_full_csv = coco_to_csv(
    DATA_ROOT / "train.json",
    OUTPUT_ROOT / "train_full.csv",
)

train_full_csv.head()


## Generar HNPs


In [ ]:
hnp_config = HNPConfig(
    checkpoint=stage1_result.best_checkpoint,
    train_csv=OUTPUT_ROOT / "train_full.csv",
    train_root=DATA_ROOT / "train",
    output_root=OUTPUT_ROOT / "hnp_patches",
    patch_size=PATCH_SIZE,
    patch_overlap=PATCH_OVERLAP,
    min_score=0.0,  # Incluir todas las detecciones
    batch_size=1,
    num_workers=NUM_WORKERS,
)


In [ ]:
print("\n" + "="*70)
print("GENERANDO HARD NEGATIVE PATCHES")
print("="*70 + "\n")

hnp_result = generate_hard_negative_patches(hnp_config)

print("\n" + "="*70)
print("HNP GENERATION COMPLETADO")
print("="*70)
print(f"✓ Parches HNP creados: {hnp_result.hnp_patches_created}")
print(f"✓ Detecciones CSV: {hnp_result.detections_csv}")
print(f"✓ Output dir: {hnp_result.output_root}")
print("="*70 + "\n")


## Combinar parches originales con HNPs para Stage 2

Copiar todos los parches originales de Stage 1 y añadir los HNPs generados.


In [ ]:
stage2_train_dir = OUTPUT_ROOT / "patches_stage2" / "train"
stage2_train_dir.mkdir(parents=True, exist_ok=True)

# Copiar parches originales de Stage 1
print("Copiando parches originales de Stage 1...")
stage1_train_dir = OUTPUT_ROOT / "patches" / "train"
copied_original = 0

for pattern in ("*.jpg", "*.JPG", "*.png", "*.PNG"):
    for src in stage1_train_dir.glob(pattern):
        dst = stage2_train_dir / src.name
        if not dst.exists():
            copy2(src, dst)
            copied_original += 1

print(f"✓ Copiados {copied_original} parches originales")

# Copiar HNPs
print("\nCopiando HNP patches...")
hnp_dir = OUTPUT_ROOT / "hnp_patches"
copied_hnp = 0

for pattern in ("*.jpg", "*.JPG", "*.png", "*.PNG"):
    for src in hnp_dir.glob(pattern):
        dst = stage2_train_dir / src.name
        if not dst.exists():
            copy2(src, dst)
            copied_hnp += 1

print(f"✓ Copiados {copied_hnp} HNP patches")
print(f"\n✓ Total Stage 2 patches: {copied_original + copied_hnp}")


**IMPORTANTE**: Para Stage 2, usamos el CSV original de Stage 1 (gt.csv), NO el gt.csv generado por HNP.

Los patches que no están en el CSV serán tratados automáticamente como background por `FolderDataset`.


In [ ]:
# Usar el GT original de stage1 (NO el de HNP)
copy2(
    OUTPUT_ROOT / "patches" / "train" / "gt.csv",
    stage2_train_dir / "gt.csv",
)

print(f"✓ CSV de Stage 2 listo: {stage2_train_dir / 'gt.csv'}")
print("  (Contiene solo anotaciones originales; HNPs son background)")


# Fase 3 — Entrenamiento Stage 2

Entrenar con los parches originales + HNPs usando una tasa de aprendizaje más baja.


In [ ]:
stage2_config = TrainConfig(
    train_root=stage2_train_dir,
    train_csv=stage2_train_dir / "gt.csv",
    val_root=OUTPUT_ROOT / "patches" / "val",
    val_csv=OUTPUT_ROOT / "patches" / "val" / "gt.csv",
    work_dir=OUTPUT_ROOT / "stage2",
    epochs=EPOCHS_STAGE2,
    batch_size=BATCH_SIZE,
    learning_rate=LR_STAGE2,
    num_workers=NUM_WORKERS,
    patch_size=PATCH_SIZE,
    stitch_overlap=STITCH_OVERLAP,
    wandb_project=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY,
    wandb_mode=WANDB_MODE,
    wandb_run_name="herdnet_stage2",
)


In [ ]:
print("\n" + "="*70)
print("INICIANDO ENTRENAMIENTO STAGE 2")
print("="*70 + "\n")

stage2_result = train_stage2(
    config=stage2_config,
    stage1_checkpoint=stage1_result.best_checkpoint,
    learning_rate=LR_STAGE2,
)

print("\n" + "="*70)
print("STAGE 2 COMPLETADO")
print("="*70)
print(f"✓ Best checkpoint: {stage2_result.best_checkpoint}")
print(f"✓ Latest checkpoint: {stage2_result.latest_checkpoint}")
print("="*70 + "\n")


# Fase 4 — Evaluación Final

Evaluar el modelo Stage 2 sobre imágenes completas de validación/test.


## Preparar CSV de test (imágenes completas)


In [ ]:
# Convertir anotaciones de test (imágenes completas) a CSV
test_csv = coco_to_csv(
    DATA_ROOT / "test.json",
    OUTPUT_ROOT / "test_full.csv",
)

test_csv.head()


## Evaluar Stage 1


In [ ]:
eval_stage1_config = EvalConfig(
    checkpoint=stage1_result.best_checkpoint,
    csv=OUTPUT_ROOT / "test_full.csv",
    root=DATA_ROOT / "test",
    output_dir=OUTPUT_ROOT / "eval_stage1",
    patch_size=PATCH_SIZE,
    overlap=STITCH_OVERLAP,
    upsample=True,
    match_radius=MATCH_RADIUS,
    batch_size=1,
    num_workers=NUM_WORKERS,
)


In [ ]:
print("\n" + "="*70)
print("EVALUANDO STAGE 1")
print("="*70 + "\n")

eval_stage1_result = evaluate_full_images(eval_stage1_config)

print("\nStage 1 Metrics:")
pd.DataFrame([eval_stage1_result.metrics["overall"]])


## Evaluar Stage 2


In [ ]:
eval_stage2_config = EvalConfig(
    checkpoint=stage2_result.best_checkpoint,
    csv=OUTPUT_ROOT / "test_full.csv",
    root=DATA_ROOT / "test",
    output_dir=OUTPUT_ROOT / "eval_stage2",
    patch_size=PATCH_SIZE,
    overlap=STITCH_OVERLAP,
    upsample=True,
    match_radius=MATCH_RADIUS,
    batch_size=1,
    num_workers=NUM_WORKERS,
)


In [ ]:
print("\n" + "="*70)
print("EVALUANDO STAGE 2")
print("="*70 + "\n")

eval_stage2_result = evaluate_full_images(eval_stage2_config)

print("\nStage 2 Metrics:")
pd.DataFrame([eval_stage2_result.metrics["overall"]])


In [ ]:
comparison = pd.DataFrame([
    {"Stage": "Stage 1", **eval_stage1_result.metrics["overall"]},
    {"Stage": "Stage 2", **eval_stage2_result.metrics["overall"]},
])

print("\n" + "="*70)
print("COMPARACIÓN STAGE 1 vs STAGE 2")
print("="*70 + "\n")

comparison


## Métricas por clase (Stage 2)


In [ ]:
per_class_df = pd.DataFrame(eval_stage2_result.metrics["per_class"]).T

print("\nMétricas por clase (Stage 2):")
per_class_df


# Resumen Final


In [ ]:
print("\n" + "="*70)
print("PIPELINE COMPLETO FINALIZADO")
print("="*70)

print("\n📁 Checkpoints:")
print(f"  Stage 1: {stage1_result.best_checkpoint}")
print(f"  Stage 2: {stage2_result.best_checkpoint}")

print("\n📊 Detecciones:")
print(f"  Stage 1: {eval_stage1_result.detections_csv}")
print(f"  Stage 2: {eval_stage2_result.detections_csv}")

print("\n📈 Métricas:")
print(f"  Stage 1 F1: {eval_stage1_result.metrics['overall']['f1_score']:.4f}")
print(f"  Stage 2 F1: {eval_stage2_result.metrics['overall']['f1_score']:.4f}")
print(f"  Mejora: {(eval_stage2_result.metrics['overall']['f1_score'] - eval_stage1_result.metrics['overall']['f1_score']):.4f}")

print("\n" + "="*70 + "\n")
